This notebook aims to build a model to predict the breast cancer status based on the histopathlogy images.

In [1]:
%matplotlib inline 

In [2]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib as plt
import matplotlib.pyplot as pyplt
import matplotlib.image as mpimg
import torch

In [3]:
import wget
import time
import os
import copy

In [14]:
data_root = "/home/ubuntu/datasets/breast-histopathology-images/"

In [15]:
!ls -l $data_root

total 3249984
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:51 10253
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:51 10254
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:51 10255
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:51 10256
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:51 10257
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:51 10258
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:51 10259
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:51 10260
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:51 10261
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:51 10262
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:51 10264
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:51 10268
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:51 10269
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:51 10272
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:51 10273
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:51 10274
drwxr-xr-x   4 ubuntu ubuntu       4096 Fe

In [16]:
!ls -l $data_root | tail

drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:52 9325
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:52 9344
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:52 9345
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:52 9346
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:52 9347
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:52 9381
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:52 9382
drwxr-xr-x   4 ubuntu ubuntu       4096 Feb  2 17:52 9383
-rw-rw-r--   1 ubuntu ubuntu 3326820824 Jan 31 15:54 breast-histopathology-images.zip
drwxr-xr-x 281 ubuntu ubuntu      12288 Feb  2 17:52 IDC_regular_ps50_idx5


In [17]:
!ls -l $data_root | wc -l

282


Seems we have the data of 279 patients. (subtracting the 3 irrelevant entries)

Let's look into the last IDC_regular-images directory

In [28]:
??os.listdir

In [18]:
files = os.listdir(data_root)
len(files)

281

In [19]:
!ls $data_root/IDC_regular_ps50_idx5/10253/

0  1


In [20]:
!ls $data_root/IDC_regular_ps50_idx5/10254/

0  1


Each patient directory has positive and negative classes

Let's check how many patches we have:

In [32]:
dirs = os.listdir(os.path.join(data_root,'IDC_regular_ps50_idx5'))
len(dirs)

279

In [37]:
total_patches = 0
for patient_directory in dirs:
    for classes in (0,1):
        patch_file_path = os.path.join(data_root,'IDC_regular_ps50_idx5',patient_directory,str(classes))
        print(len(patch_file_path))
        patch_images = os.listdir(patch_file_path)
        total_patches += len(patch_images)

80
80
80
80
79
79
79
79
80
80
79
79
79
79
80
80
80
80
79
79
80
80
80
80
79
79
80
80
80
80
80
80
80
80
80
80
79
79
79
79
80
80
79
79
80
80
79
79
80
80
79
79
80
80
79
79
80
80
79
79
80
80
79
79
80
80
80
80
80
80
79
79
79
79
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
79
79
80
80
79
79
79
79
79
79
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
79
79
79
79
80
80
80
80
79
79
80
80
80
80
80
80
80
80
79
79
80
80
80
80
79
79
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
79
79
80
80
80
80
80
80
80
80
80
80
80
80
80
80
79
79
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
79
79
80
80
80
80
79
79
79
79
79
79
79
79
79
79
80
80
80
80
79
79
79
79
79
79
80
80
80
80
79
79
80
80
80
80
80
80
80
80
80
80
80
80
79
79
79
79
80
80
80
80
80
80
80
80
80
80
80
80
80
80
79
79
80
80
80
80
80
80
79
79
79
79
80
80
80
80
80
80
80
80
80
80
79
79
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
79
79
79
79
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
8

In [36]:
total_patches

277524

In [38]:
277524/279

994.7096774193549